In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
xrange = range
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import os

#pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', 200)

print("The working directory was: {0}".format(os.getcwd()))
os.chdir("C:/hjBoralex/etl/gitcwd")
print("The current working directory is: {0}".format(os.getcwd()))

The working directory was: C:\hjBoralex\etl
The current working directory is: C:\hjBoralex\etl\gitcwd


In [32]:
mypath = "C:/Users/hermann.ngayap/Desktop/data/"
marketpath = "C:/Users/hermann.ngayap/Desktop/data/market/markets/"

## P50_P90_ASSET PLANIF

In [47]:
df = pd.read_excel(marketpath + "template_asset_vmr_planif.xlsx")
df = df.loc[(df["en_planif"] == "Oui") & (df["technologie"] == "éolien")]
mean_perc = pd.read_excel(marketpath + "template_prod_id.xlsx", sheet_name="mean_perc")
print(df.shape)

(92, 15)


In [48]:
#df = pd.read_excel(marketpath + "template_asset_planif.xlsx")
df1 = pd.read_excel(marketpath + "template_asset_vmr_planif.xlsx")
df1 = df1.loc[df1["en_planif"] == "Oui"]


#To fill n/a with 0.80
df1["taux_succès"].fillna(0.80, inplace=True)

#To calculate mw 
df1["puissance_installée"] = df1["mw"] * df1["taux_succès"]
df1["date_merchant"].fillna(df1["cod"] + pd.DateOffset(years=20), inplace=True) 

#To select only data with 2023 cod 
filt = df1['cod'] > dt.datetime.today().strftime('%Y-%m-%d') 
df1 = df1.loc[filt]

#df1["puissance_installée"] = df1["mw"] * df1["taux_succès"]

df1.loc[df1['technologie'] == 'éolien', 'p50'] = df1["puissance_installée"] * 8760 * 0.25#calculer le p_50 projet éolien en planification
df1.loc[df1['technologie'] == 'éolien', 'p90'] = df1["puissance_installée"] * 8760 * 0.20#calculer le p_90 projet éolien en planification

df1.loc[df1['technologie'] == 'solaire', 'p50'] = df1["puissance_installée"] * 8760 * 0.15#calculer le p_50 projet solaire en planification
df1.loc[df1['technologie'] == 'solaire', 'p90'] = df1["puissance_installée"] * 8760 * 0.13#calculer le p_90 projet solaire en planification


prod_planif = df1.loc[:,["rw_id", "asset_id", "projet_id", "projet", "technologie", "cod","p50", "p90"]]

prod_planif_solaire = prod_planif.loc[prod_planif['technologie'] == "solaire"]
prod_planif_eolien = prod_planif.loc[prod_planif['technologie'] == "éolien"]

prod_planif_solaire.reset_index(drop = True, inplace=True)
prod_planif_eolien.reset_index(drop = True, inplace=True)

nbr_sol = len(prod_planif_solaire) 
nbr_eol = len(prod_planif_eolien) 

prod_planif_solaire.to_excel(marketpath + "prod_planif_solaire.xlsx", index = False)
prod_planif_eolien.to_excel(marketpath + "prod_planif_eolien.xlsx", index = False)

mean_perc_sol = mean_perc.iloc[:,[0, 1]]
mean_perc_eol = mean_perc.iloc[:,[0,-1]]

#create a dataframe with date from 2022 to 2028 solaire
start_date = pd.to_datetime(["01-01-2022"] * nbr_sol)
d1 = pd.DataFrame()
for i in range(0,84):
    df_buffer = prod_planif_solaire.copy() 
    df_buffer["date"] = start_date
    d1 = pd.concat([d1, df_buffer],axis=0)
    start_date = start_date + pd.DateOffset(months=1)
    
#reset index    
d1.reset_index(drop=True, inplace=True)
#create a mth column containing number of month
xrange = range
mean_perc_sol = mean_perc_sol.assign(mth=[1 + i for i in xrange(len(mean_perc_sol))])[['mth'] + mean_perc_sol.columns.tolist()]

#To calculate p50 and p90 solar
s = mean_perc_sol.set_index('mth')['m_pct_solaire']
pct = pd.to_datetime(d1['date']).dt.month.map(s)
d1['p50_adj'] = d1['p50'] * pct
d1['p90_adj'] = d1['p90'] * pct

#create a dataframe with date from 2022 to 2028 éolien
start_date = pd.to_datetime(["01-01-2022"] * nbr_eol)
d2 = pd.DataFrame()
for i in range(0,84):
    df_buffer = prod_planif_eolien.copy() 
    df_buffer["date"] = start_date
    d2 = pd.concat([d2, df_buffer],axis=0)
    start_date = start_date + pd.DateOffset(months=1)

#reset index    
d2.reset_index(drop=True, inplace=True)
#create a mth column containing number of month
mean_perc_eol = mean_perc_eol.assign(mth=[1 + i for i in xrange(len(mean_perc_eol))])[['mth'] + mean_perc_eol.columns.tolist()]

#To calculate p50 and p90 eolien
s2 = mean_perc_eol.set_index('mth')['m_pct_eolien']
pct = pd.to_datetime(d2['date']).dt.month.map(s2)
d2['p50_adj'] = d2['p50'] * pct
d2['p90_adj'] = d2['p90'] * pct

#To create new columns année et mois
d1["cod"] = pd.to_datetime(d1["cod"])
d1["date"] = pd.to_datetime(d1["date"])
d1['année'] = d1['date'].dt.year
d1['mois'] = d1['date'].dt.month
d1 = d1[['asset_id','projet_id', 'projet', 'cod', 'date', 'année', 'mois', 'p50_adj', 'p90_adj']]

#To remove p50 p90 based on date_cod
#Condition:date column is less (in total seconds) than first date for each projet_id's first date_cod value
cond=((d1['date'] - d1.groupby('projet_id')['cod'].transform('first')).dt.total_seconds())<0
d1['p50_adj'] = np.where(cond,'', d1['p50_adj'])
d1['p90_adj'] = np.where(cond,'', d1['p90_adj'])
#To remove p50 p90 based on date_dementelement
#cond_2=((d1['date'] - d1.groupby('projet_id')['date_dementelement'].transform('first')).dt.total_seconds())>0
#d1['p50_adj'] = np.where(cond_2, '', d1['p50_adj'])
#d1['p90_adj'] = np.where(cond_2, '', d1['p90_adj'])

#To reset index
d1.reset_index(inplace=True, drop=True)
d1 = d1.assign(rw_id=[1 + i for i in xrange(len(d1))])[['rw_id'] + d1.columns.tolist()]
d1 = d1[['rw_id', 'asset_id', 'projet_id', 'projet', 'année', 'mois', 'p50_adj', 'p90_adj']]


#To create new columns année et mois
d2["cod"] = pd.to_datetime(d2["cod"])
d2["date"] = pd.to_datetime(d2["date"])
d2['année'] = d2['date'].dt.year
d2['mois'] = d2['date'].dt.month
d2 = d2[['asset_id','projet_id', 'projet', 'cod', 'date', 'année', 'mois', 'p50_adj', 'p90_adj']]

#To remove p50 p90 based on date_cod
#Condition:date column is less (in total seconds) than first date for each projet_id's first date_cod value
cond=((d2['date'] - d2.groupby('projet_id')['cod'].transform('first')).dt.total_seconds())<0
d2['p50_adj'] = np.where(cond,'', d2['p50_adj'])
d2['p90_adj'] = np.where(cond,'', d2['p90_adj'])
#To remove p50 p90 based on date_dementelement
#cond_2=((d2['date'] - d2.groupby('projet_id')['date_dementelement'].transform('first')).dt.total_seconds())>0
#d2['p50_adj'] = np.where(cond_2, '', d2['p50_adj'])
#d2['p90_adj'] = np.where(cond_2, '', d2['p90_adj'])

#To reset index
d2.reset_index(inplace=True, drop=True)
d2 = d2.assign(rw_id=[1 + i for i in xrange(len(d2))])[['rw_id'] + d2.columns.tolist()]
d2 = d2[['rw_id', 'asset_id', 'projet_id', 'projet', 'année', 'mois', 'p50_adj', 'p90_adj']]

#To export as excel files
d1.to_excel(marketpath + 'p50_p90_planif_solaire.xlsx', index=False, float_format="%.5f")
d2.to_excel(marketpath + 'p50_p90_planif_eolien.xlsx', index=False, float_format="%.5f")

In [52]:
print(prod_planif.loc[prod_planif['projet_id'] == 'APV1'])
df_temp = d1.loc[d1['projet_id'] == 'APV1']
df_temp.tail(20)

    rw_id  asset_id projet_id     projet technologie        cod         p50  \
96     97        97      APV1  Agri-PV 1     solaire 2027-09-01  1553.06916   

            p90  
96  1345.993272  


,rw_id,asset_id,projet_id,projet,année,mois,p50_adj,p90_adj
6720,6721,97,APV1,Agri-PV 1,2027,5,,
6825,6826,97,APV1,Agri-PV 1,2027,6,,
6930,6931,97,APV1,Agri-PV 1,2027,7,,
7035,7036,97,APV1,Agri-PV 1,2027,8,,
7140,7141,97,APV1,Agri-PV 1,2027,9,141.950521224,123.02378506080001
7245,7246,97,APV1,Agri-PV 1,2027,10,106.6647899088,92.44281792096002
7350,7351,97,APV1,Agri-PV 1,2027,11,67.80699952559999,58.76606625552
7455,7456,97,APV1,Agri-PV 1,2027,12,59.63785574399999,51.6861416448
7560,7561,97,APV1,Agri-PV 1,2028,1,65.6327027016,56.88167567472001
7665,7666,97,APV1,Agri-PV 1,2028,2,86.9408115768,75.34870336656002


#### MERGING p50_p90_asset_vmr & p50_p90_planif_solaire & p50_p90_planif_eolien

In [63]:
df_1=pd.read_excel(marketpath + 'p50_p90_asset_vmr.xlsx')
df_2=pd.read_excel(marketpath + 'p50_p90_planif_solaire.xlsx')
df_3=pd.read_excel(marketpath + 'p50_p90_planif_eolien.xlsx')


frames=[df_1, df_2, df_3]
p50_p90_vmr_planif =pd.concat(frames, axis= 0, ignore_index=True)
p50_p90_vmr_planif.drop("rw_id", axis=1, inplace=True)
p50_p90_vmr_planif = p50_p90_vmr_planif.assign(rw_id=[1 + i for i in xrange(len(p50_p90_vmr_planif))])[['rw_id'] + p50_p90_vmr_planif.columns.tolist()]

In [64]:
p50_p90_vmr_planif.tail()

,rw_id,asset_id,projet_id,projet,année,mois,p50_adj,p90_adj
24607,24608,286,GO19,Repowering Ronchois,2028,12,9828.541296,7862.833037
24608,24609,289,SO14,Bruyeres - Glenic,2028,12,3033.500400,2426.800320
24609,24610,290,GO10,Repowering Grand-Camp,2028,12,2426.800320,1941.440256
24610,24611,292,NGE9,Repowering St-Patrick Valhuon,2028,12,5678.712749,4542.970199
24611,24612,293,NGE19,XLCB 2,2028,12,1801.899238,1441.519390


In [65]:
#To export as excel file
p50_p90_vmr_planif.to_excel(marketpath + 'p50_p90_asset_vmr_planif.xlsx', index=False, float_format="%.5f")